# Практическое задание №8 по теме "Рекуррентные нейронные сети RNN LSTM GRU".

In [1]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 569.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 45.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=8b094cb7cfd3e267572378b61666ef6656d0ee8d5ca4335583a5c7f9d1f2c2c5
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


### Загрузка библиотек:

In [2]:
import pandas as pd
import numpy as np
from string import punctuation
from pymorphy2 import MorphAnalyzer
import re

from gensim.models import KeyedVectors
import gensim
from tensorflow.keras.layers import Embedding

import keras
from keras.models import Sequential, Model
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping  
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking

import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Загрузка данных:

In [3]:
df_train = pd.read_csv("/content/drive/MyDrive/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/test.csv")
df_val = pd.read_csv("/content/drive/MyDrive/val.csv")

In [4]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


### Препроцессинг данных:

In [5]:
sw = set(stopwords.words('russian'))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [6]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [7]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [8]:
y_train = df_train['class'].values
y_val = df_val['class'].values

### Модель RNN:

In [9]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 64s 191ms/step - loss: 0.5601 - accuracy: 0.7007 - val_loss: 0.4899 - val_accuracy: 0.7571
Epoch 2/10
319/319 [==============================] - 58s 181ms/step - loss: 0.2938 - accuracy: 0.8789 - val_loss: 0.5611 - val_accuracy: 0.7455


In [11]:
score0 = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)

45/45 [==============================] - 1s 22ms/step - loss: 0.5764 - accuracy: 0.7398


### Модель CNN:

In [12]:
max_words = 200
max_len = 40
num_classes = 1
epochs = 20
batch_size = 512
print_batch_n = 100

In [13]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

tokens = word_tokenize(train_corpus)

tokens_filtered = [word for word in tokens if word.isalnum()]

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [14]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [15]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [16]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

num_classes = 2
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_val = keras.utils.to_categorical(df_val["class"], num_classes)

Создание модели:

In [17]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [18]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Обучение модели:

In [19]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
319/319 [==============================] - 61s 188ms/step - loss: 0.6259 - accuracy: 0.6186 - val_loss: 0.6167 - val_accuracy: 0.6286
Epoch 2/20
319/319 [==============================] - 62s 195ms/step - loss: 0.6129 - accuracy: 0.6325 - val_loss: 0.6155 - val_accuracy: 0.6335
Epoch 3/20
319/319 [==============================] - 61s 192ms/step - loss: 0.6077 - accuracy: 0.6381 - val_loss: 0.6139 - val_accuracy: 0.6342
Epoch 4/20
319/319 [==============================] - 59s 184ms/step - loss: 0.6015 - accuracy: 0.6443 - val_loss: 0.6135 - val_accuracy: 0.6353
Epoch 5/20
319/319 [==============================] - 60s 187ms/step - loss: 0.5943 - accuracy: 0.6506 - val_loss: 0.6159 - val_accuracy: 0.6305


Результаты:

In [20]:
score1 = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('CNN:')
print('Test score:', score1[0])
print('Test accuracy:', score1[1])
print("-"*25)
print('RNN:')
print('Test score:', score0[0])
print('Test accuracy:', score0[1])

45/45 [==============================] - 2s 44ms/step - loss: 0.6217 - accuracy: 0.6274
CNN:
Test score: 0.6217420101165771
Test accuracy: 0.6273861527442932
-------------------------
RNN:
Test score: 0.5764308571815491
Test accuracy: 0.7397610545158386


**Вывод:** по общему скору рекурентная сеть немного проигрывает сверточной, но по точности значительно ее превосходит. 

---